In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error


In [2]:
weather_data = np.load('Dataset/inputs_weather_train.npy', allow_pickle=False, fix_imports=True)
other_data = np.load('Dataset/inputs_others_train.npy', allow_pickle=False, fix_imports=True)

yield_data = np.load('Dataset/yield_train.npy', allow_pickle=False, fix_imports=True)
test_weather_data = np.load('Dataset/inputs_weather_test.npy', allow_pickle=False, fix_imports=True)
test_other_data = np.load('Dataset/inputs_others_test.npy', allow_pickle=False, fix_imports=True)


In [3]:
other_data

array([['3.0', '243.0', '"KS"', '2009.0', '98.0'],
       ['6.0', '114.0', '"AL"', '2013.0', '16.0'],
       ['4.0', '2878.0', '"TN"', '2015.0', '72.0'],
       ...,
       ['2.0', '182.0', '"IN"', '2006.0', '150.0'],
       ['5.0', '699.0', '"LA"', '2008.0', '1.0'],
       ['8.0', '1416.0', '"GA"', '2004.0', '7.0']], dtype='<U32')

In [4]:
encoder = LabelEncoder()
other_data[:, 2] = encoder.fit_transform(other_data[:, 2].reshape(-1, 1))
test_other_data[:, 2] = encoder.fit_transform(test_other_data[:, 2].reshape(-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
for index in range(len(weather_data)):
    weather_data[index] = scaler.fit_transform(weather_data[index])
other_data = scaler.fit_transform(other_data)
for index in range(len(test_weather_data)):
    test_weather_data[index] = scaler.fit_transform(test_weather_data[index])
test_other_data = scaler.fit_transform(test_other_data)
yield_data = scaler.fit_transform(yield_data.reshape(-1, 1)).flatten()


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [5]:
trainX = []
lenth = len(weather_data)
for i in range(lenth):
    weather = weather_data[i]
    weather = weather.flatten()
    other = other_data[i]
    temp = np.hstack((weather, other)) 
    trainX.append(temp)
trainX = np.array(trainX)
trainY = yield_data[:lenth]

testX = []
for i in range(len(test_weather_data)):
    test_weather = test_weather_data[i]
    test_weather = test_weather.flatten()
    test_other = test_other_data[i]
    temp = np.hstack((test_weather, test_other))
    testX.append(temp)
testX = np.array(testX)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)


(93028, 1503)
(93028,)
(10337, 1503)


In [6]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
import math



In [7]:
look_back = len(trainX[0])
print(look_back)
model = Sequential()
model.add(LSTM(100, input_shape=(1, look_back), return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()


1503
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 100)            641600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 32)             17024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 666,977
Trainable params: 666,977
Non-trainable params: 0
_________________________________________________________________


In [8]:
trainX = np.reshape(trainX, (len(trainX), 1, len(trainX[0])))
testX = np.reshape(testX, (len(testX), 1, len(testX[0])))
print(testX.shape)
print(trainX.shape)

(10337, 1, 1503)
(93028, 1, 1503)


In [ ]:
model.fit(trainX, trainY, epochs=600, batch_size=240, verbose=1)


Epoch 1/600
388/388 [==============================] - 8s 11ms/step - loss: 0.0213
Epoch 2/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0120
Epoch 3/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0101
Epoch 4/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0092
Epoch 5/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0087
Epoch 6/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0081
Epoch 7/600
388/388 [==============================] - 4s 10ms/step - loss: 0.0078
Epoch 8/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0074
Epoch 9/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0071
Epoch 10/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0069
Epoch 11/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0067
Epoch 12/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0066
Epoch 13/60

388/388 [==============================] - 3s 9ms/step - loss: 0.0051
Epoch 100/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0051
Epoch 101/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0052
Epoch 102/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0051
Epoch 103/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0051
Epoch 104/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0051
Epoch 105/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0052
Epoch 106/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0050
Epoch 107/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0051
Epoch 108/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0051
Epoch 109/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0051
Epoch 110/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0050
Epoch 

388/388 [==============================] - 3s 9ms/step - loss: 0.0048
Epoch 197/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0048
Epoch 198/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0048
Epoch 199/600
388/388 [==============================] - 4s 10ms/step - loss: 0.0048
Epoch 200/600
388/388 [==============================] - 4s 10ms/step - loss: 0.0048
Epoch 201/600
388/388 [==============================] - 4s 10ms/step - loss: 0.0048
Epoch 202/600
388/388 [==============================] - 4s 10ms/step - loss: 0.0048
Epoch 203/600
388/388 [==============================] - 4s 10ms/step - loss: 0.0047
Epoch 204/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0048
Epoch 205/600
388/388 [==============================] - 4s 10ms/step - loss: 0.0047
Epoch 206/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0048
Epoch 207/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0047


388/388 [==============================] - 3s 9ms/step - loss: 0.0045
Epoch 294/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0045
Epoch 295/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0045
Epoch 296/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0045
Epoch 297/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0045
Epoch 298/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0046
Epoch 299/600
388/388 [==============================] - 4s 9ms/step - loss: 0.0045
Epoch 300/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0045
Epoch 301/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0045
Epoch 302/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0045
Epoch 303/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0045
Epoch 304/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0045
Epoch 

388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 391/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 392/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 393/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 394/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 395/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 396/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 397/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 398/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 399/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 400/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 401/600
388/388 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 

388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 488/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 489/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 490/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 491/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 492/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 493/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 494/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 495/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 496/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 497/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 498/600
388/388 [==============================] - 3s 9ms/step - loss: 0.0043
Epoch 

In [9]:
trainPredict = model.predict(trainX)
print(trainY.shape)
print(trainPredict.shape)

(93028,)
(93028, 1)


In [10]:
trainPredict = scaler.inverse_transform(trainPredict).flatten()
print(trainY.shape)
trainY = scaler.inverse_transform([trainY]).flatten()
print(trainY.shape)
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print("RMSE for train data is {}".format(trainScore))


(93028,)
(93028,)
RMSE for train data is 7.425513827873115


In [11]:
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict).flatten()
print(len(testPredict))
# testY = scaler.inverse_transform([testY]).flatten()
# testScore = math.sqrt(mean_squared_error(testY, testPredict))
np.save('Dataset/yield_predict.npy', testPredict"{:.2f}".format(float), allow_pickle=False, fix_imports=True)
testPredict

10337


array([44.214634, 59.02436 , 58.10894 , ..., 47.988777, 55.68764 ,
       64.26153 ], dtype=float32)